# SQL_ONE - Análise de Dados com SQL
## Aula 4 - Group by, First Value, Last Value, Over, Row Number, Qualify, Regex Replace, Translate, Function

### 1 - Base de dados pilotos
#### Acessando a base de dados com PANDAS, criando dataframe com SPARK, salvando numa TABLE para ler em SQL.

In [0]:
%py
# 1 - Importando o pandas
import pandas as pd

# Acessando a base de dados df_pilotos
df_pilotos = pd.read_csv("https://raw.githubusercontent.com/wallacecarlis/arquivos/main/todos_pilotos.csv")

# Criando um spark dataframe
df_pyspark_pilotos = spark.createDataFrame(df_pilotos)

# Criando uma tabela delta
df_pyspark_pilotos.write.format("delta").mode("overwrite").saveAsTable("df_pilotos")

### 2 - Select
#### Visualizando a base de dados com SELECT.

In [0]:
-- 2 - Visualizando os dados da tabela df_pilotos
SELECT * FROM df_pilotos;

nome,idade,naturalidade,equipe,pais
Ayrton Senna,62,São Paulo,Toleman,Brasil
Emerson Fittipaldi,76,São Paulo,Lotus,Brasil
José Carlos Pace,78,São Paulo,Willys,Brasil
Lewis Hamilton,37,Stevenage,McLaren,Inglaterra
Max Verstappen,25,Hasselt,Toro Rosso,Bélgica
Michael Schumacher,53,Hürth,Jordan,Alemanha
Nelson Piquet,70,Rio de Janeiro,Ensign,Brasil
Rubens Barrichello,50,São Paulo,Jordan,Brasil
Jos Verstappen,50,Montfort,Bennetton,Holanda
Sebastian Vettel,35,Heppenhein,Sauber,Alemanha


### 3 - Group by, count
#### Uma das principais técnicas utilizadas no cotidiano do profissional de dados é o agrupamento, fazemos isso com o GROUP BY. No caso, verificaremos a quantidade de pilotos por país usando o COUNT.

In [0]:
-- 3 - Visualizando a quantidade de pilotos por país
SELECT
  pais,
  COUNT(pais)
FROM df_pilotos
GROUP BY pais;

pais,count(pais)
Inglaterra,3
Alemanha,3
Bélgica,1
Brasil,5
Holanda,1


### 4 - Group By, Count, Order By
#### Após realizar o agrupamento com GROUP BY, podemos ordenar o resultado com o ORDER BY.

In [0]:
-- 4 - Visualizando a quantidade de pilotos por país e naturalidade
SELECT
  pais,
  naturalidade,
  COUNT(pais)
FROM df_pilotos
GROUP BY pais, naturalidade
ORDER BY pais;

pais,naturalidade,count(pais)
Alemanha,Hürth,1
Alemanha,Wiesbaden,1
Alemanha,Heppenhein,1
Brasil,Rio de Janeiro,1
Brasil,São Paulo,4
Bélgica,Hasselt,1
Holanda,Montfort,1
Inglaterra,Upton-upon-Severn,1
Inglaterra,Frome,1
Inglaterra,Stevenage,1


### 5 - Sum
#### Podemos utilizar a soma como forma de agregação com SUM.

In [0]:
-- 5 - Visualizando soma da idade dos pilotos por país
SELECT
  pais,
  SUM(idade) AS soma_idade
FROM df_pilotos
GROUP BY pais
ORDER BY pais;

pais,soma_idade
Alemanha,125
Brasil,336
Bélgica,25
Holanda,50
Inglaterra,148


### 6 - Group by, Having
#### O HAVING funciona como um filtro para o GROUP BY. Nele podemos passar a função de agregação que criamos no SELECT.

In [0]:
-- 6 - Visualizando soma da idade dos pilotos por país
SELECT
  pais,
  SUM(idade) AS soma_idade
FROM df_pilotos
GROUP BY pais
HAVING SUM(idade) >= 100
ORDER BY pais;

pais,soma_idade
Alemanha,125
Brasil,336
Inglaterra,148


In [0]:
-- 7 - Visualizando média da idade dos pilotos por país
SELECT
  pais,
  ROUND(AVG(idade), 0) AS media_idade
FROM df_pilotos
GROUP BY pais
HAVING media_idade >= 50
ORDER BY pais;

pais,media_idade
Brasil,67.0
Holanda,50.0


In [0]:
-- 8 - Visualizando soma e média de idade dos pilotos por país e naturalidade
SELECT
  pais,
  naturalidade,
  SUM(idade) AS soma_idade,
  AVG(idade) AS media_idade
FROM df_pilotos
GROUP BY pais, naturalidade
HAVING naturalidade IN ('São Paulo', 'Rio de Janeiro') OR media_idade >= 50
ORDER BY pais;

pais,naturalidade,soma_idade,media_idade
Alemanha,Hürth,53,53.0
Brasil,Rio de Janeiro,70,70.0
Brasil,São Paulo,266,66.5
Holanda,Montfort,50,50.0
Inglaterra,Upton-upon-Severn,69,69.0


### 7 - Distinct, first_value, Over
#### Podemos ordenar os dados para trazer somente o primeiro piloto de cada país utilizando FIRST VALUE, OVER, PARTITION BY e GROUP BY.

In [0]:
-- 9 - Visualizando o primeiro pilotos de cada país com first_value
SELECT 
  DISTINCT pais,
  FIRST_VALUE(nome) 
    OVER (PARTITION BY pais 
          ORDER BY pais) AS primeiro_piloto
FROM
    df_pilotos;

pais,primeiro_piloto
Alemanha,Michael Schumacher
Brasil,Ayrton Senna
Bélgica,Max Verstappen
Holanda,Jos Verstappen
Inglaterra,Lewis Hamilton


### 8 - Distinct, last_value, Over
#### De igual modo, podemos trazer o último piloto de cada país, no caso, precisaremos de LAST VALUE, OVER, PARTITION BY, ORDER BY, ROWS BETWEEN.

In [0]:
-- 10 - Visualizando o último piloto de cada país com last_value
SELECT 
  DISTINCT pais,
  LAST_VALUE(nome) 
    OVER (PARTITION BY pais 
          ORDER BY pais 
          ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS ultimo_piloto
FROM
    df_pilotos;

pais,ultimo_piloto
Alemanha,Nico Rosberg
Brasil,Rubens Barrichello
Bélgica,Max Verstappen
Holanda,Jos Verstappen
Inglaterra,Jenson Button


### 9 - Row Number
#### Em alguns momentos precisaremos criar uma coluna para estabelecer um ranking, fazemos isso com o ROW NUMBER, OVER, PARTITION BY e ORDER BY.

In [0]:
-- 11 - Visualizando a ordem dos pilotos por país com row number
SELECT
  pais,
  nome,
  ROW_NUMBER() 
    OVER (PARTITION BY pais 
          ORDER BY pais) AS ordem_piloto_por_pais
FROM df_pilotos;

pais,nome,ordem_piloto_por_pais
Alemanha,Michael Schumacher,1
Alemanha,Sebastian Vettel,2
Alemanha,Nico Rosberg,3
Brasil,Ayrton Senna,1
Brasil,Emerson Fittipaldi,2
Brasil,José Carlos Pace,3
Brasil,Nelson Piquet,4
Brasil,Rubens Barrichello,5
Bélgica,Max Verstappen,1
Holanda,Jos Verstappen,1


### 10 - Distinct*, subquery, row_number, over, where
#### Em determinados momentos, precisaremos definir a ordem do piloto a ser retornada, contanto que não seja o primeiro, já que temos o FIRST VALUE, ou o último, pois temos o LAST VALUE. Sendo assim, podemos utilizar uma SUBQUERY com ROW NUMBER, OVER, PARTITION BY, ORDER BY e WHERE.

In [0]:
-- 12 - Visualizando piloto por ordem específica com subquery e row number
SELECT 
  DISTINCT pais,
  nome AS segundo_piloto
    FROM (SELECT
          pais,
          nome,
          ROW_NUMBER() 
            OVER (PARTITION BY pais 
                  ORDER BY pais) AS ordem_piloto_por_pais
          FROM df_pilotos) AS tabela_row_number
    WHERE ordem_piloto_por_pais = 2;

pais,segundo_piloto
Alemanha,Sebastian Vettel
Brasil,Emerson Fittipaldi
Inglaterra,Nigel Mansell


### 11 - QUALIFY, ROW NUMBER, OVER, PARTITION BY
#### Podemos repetir o exemplo anterior de uma forma mais simples com QUALIFY, ROW NUMBER, OVER, PARTITION BY.

In [0]:
-- 13 - Visualizando piloto por ordem específica com qualify e row number
SELECT
    pais,
    nome AS segundo_piloto
FROM df_pilotos
QUALIFY ROW_NUMBER() OVER (PARTITION BY pais ORDER BY pais) = 2;

pais,segundo_piloto
Alemanha,Sebastian Vettel
Brasil,Emerson Fittipaldi
Inglaterra,Nigel Mansell


### 12 - Regex_replace, translate, function
#### Para tratamentos específicos de texto, por exemplo, podemos utilizar o REGEX REPLACE, TRANSLATE, ou simplesmente podemos criar nossa própria função com CREATE FUNCTION.

In [0]:
-- 14 - Retirando acentos com regex replace
SELECT regexp_replace('São Paulo', '[ãáàâ]', 'a') AS Regex_Replace;

Regex_Replace
Sao Paulo


In [0]:
-- 15 - Retirando acentos com translate
SELECT translate('São Paulo', 'ãáàâéêíóôõúüç', 'aaaaeeiooouuc') AS Translate;

Translate
Sao Paulo


In [0]:
-- 16 - Criando função para remover acentos com translate
DROP FUNCTION IF EXISTS remove_acentos;

CREATE FUNCTION remove_acentos(input STRING)
RETURNS STRING
RETURN translate(input, 
                'áàãâéêíóôõúüçÁÀÃÂÉÊÍÓÔÕÚÜÇ', 
                'aaaaeeiooouucAAAAEEIOOOUUC');

In [0]:
-- 17 - Aplicando função para remover acentos
SELECT
  remove_acentos(nome) AS nome,
  idade,
  remove_acentos(naturalidade) AS naturalidade,
  equipe
FROM df_pilotos;

nome,idade,naturalidade,equipe
Ayrton Senna,62,Sao Paulo,Toleman
Emerson Fittipaldi,76,Sao Paulo,Lotus
Jose Carlos Pace,78,Sao Paulo,Willys
Lewis Hamilton,37,Stevenage,McLaren
Max Verstappen,25,Hasselt,Toro Rosso
Michael Schumacher,53,Hurth,Jordan
Nelson Piquet,70,Rio de Janeiro,Ensign
Rubens Barrichello,50,Sao Paulo,Jordan
Jos Verstappen,50,Montfort,Bennetton
Sebastian Vettel,35,Heppenhein,Sauber


### 13 - Create Function
#### Podemos criar nossa função utilizando algumas funções já existentes como DECIMAL e CAST.

In [0]:
-- 18 - Criando função para converter inteiro para decimal com cast
DROP FUNCTION IF EXISTS inteiro_para_decimal;

CREATE FUNCTION inteiro_para_decimal(valor INT)
RETURNS DECIMAL(10,2)
RETURN CAST(valor AS DECIMAL(10,2));

In [0]:
-- 19 - Aplicando funções para remover acentos e transformar números inteiros para decimais
SELECT
  remove_acentos(nome) AS nome,
  inteiro_para_decimal(idade) AS idade,
  remove_acentos(naturalidade) AS naturalidade,
  equipe
FROM df_pilotos;

nome,idade,naturalidade,equipe
Ayrton Senna,62.00,Sao Paulo,Toleman
Emerson Fittipaldi,76.00,Sao Paulo,Lotus
Jose Carlos Pace,78.00,Sao Paulo,Willys
Lewis Hamilton,37.00,Stevenage,McLaren
Max Verstappen,25.00,Hasselt,Toro Rosso
Michael Schumacher,53.00,Hurth,Jordan
Nelson Piquet,70.00,Rio de Janeiro,Ensign
Rubens Barrichello,50.00,Sao Paulo,Jordan
Jos Verstappen,50.00,Montfort,Bennetton
Sebastian Vettel,35.00,Heppenhein,Sauber


### 14 - Update
#### Podemos atualizar valores utilizando a função que criamos. Fazemos isso com UPDATE e a função criada.

In [0]:
-- 20 - Definindo tipo de dado da coluna
UPDATE df_pilotos
SET idade = inteiro_para_decimal(idade);

num_affected_rows
13
